In [63]:
def popcount(x: int) -> int:
    return bin(x).count('1')



class Term:
    def __init__(self, val: int, mask: int):
        self.marked = False
        self.val = val
        self.mask = mask

    def can_merge(self, other: 'Term') -> bool:
        return self.mask == other.mask and popcount(self.val ^ other.val) == 1

    def merge(self, other: 'Term') -> 'Term':
        if not self.can_merge(other):
            raise ValueError("Cannot merge terms")

        return Term(self.val & other.val, self.mask & ~(self.val ^ other.val))

    def mark(self):
        self.marked = True

    def print(self, header: str = ""):
        print(f"{header}{self.val:02x} & {self.mask:02x} ({'v' if self.marked else '-'})")

    def __eq__(self, other: 'Term') -> bool:
        return self.val == other.val and self.mask == other.mask

    def __hash__(self) -> int:
        return hash((self.val, self.mask))



class GroupedTerms:
    def __init__(self, active_bits: int, terms: list[Term]):
        self.active_bits = active_bits
        self.terms = terms

    def print(self, header: str = ""):
        print(f"{header}{self.active_bits} bits:")
        for term in self.terms:
            term.print(header + "  ")

    def merge(self, other: 'GroupedTerms') -> 'GroupedTerms':
        if self.active_bits + 1 != other.active_bits:
            raise ValueError("Cannot merge groups")

        new_terms: list[Term] = []

        for term1 in self.terms:
            for term2 in other.terms:
                if term1.can_merge(term2):
                    term1.mark()
                    term2.mark()
                    new_terms.append(term1.merge(term2))

        return GroupedTerms(self.active_bits, new_terms)

    def reduce_duplicates(self):
        self.terms = list(set(self.terms))



class TermGroup:
    def __init__(self, groups: list[GroupedTerms]):
        self.groups = sorted(groups, key=lambda x: x.active_bits)
        self.confirmed_terms: list[Term] = []

    def print(self, header: str = ""):
        print(f"{header}Groups:")
        for group in self.groups:
            group.print(header + "  ")
        print(f"{header}Confirmed terms:")
        for term in self.confirmed_terms:
            term.print(header + "  ")

    def reduce_duplicates(self):
        self.confirmed_terms = list(set(self.confirmed_terms))

        for group in self.groups:
            group.reduce_duplicates()

    def merge(self) -> 'TermGroup':
        new_groups: list[GroupedTerms] = []

        for group_1 in self.groups:
            group_2_candidates = [
                group_2
                for group_2 in self.groups
                if group_2.active_bits == group_1.active_bits + 1
            ]
            if len(group_2_candidates) == 0:
                continue

            group_2 = group_2_candidates[0]
            new_groups.append(group_1.merge(group_2))



        new_group = TermGroup(new_groups)

        non_marked_terms = [
            term
            for group in self.groups
            for term in group.terms
            if not term.marked
        ]

        new_group.confirmed_terms = self.confirmed_terms + non_marked_terms

        new_group.reduce_duplicates()

        return new_group

    def merge_all(self) -> 'TermGroup':
        new_group = self
        while True:
            new_group = new_group.merge()
            if len(new_group.groups) == 0:
                break

        return new_group


terms_raw = [
    0x00, 0x01,
    0x20, 0x21, 0x22,
    0x30, 0x31, 0x32, 0x38,
    # 0x40, 0x41, 0x42,

    0xfe, 0xff
]

groups = TermGroup([
    GroupedTerms(active_bits,[
        Term(x, 0xff)
        for x in terms_raw
        if popcount(x) == active_bits
    ])
    for active_bits in set([popcount(x) for x in terms_raw])
])

groups = groups.merge_all()
ws_filter = []
ws_filter += ["robo"]
for term in groups.confirmed_terms:
    ws_filter.append(f"robo.opcode & 0x{term.mask:02x} != 0x{term.val:02x}")

print(" and ".join(ws_filter))

robo and robo.opcode & 0xde != 0x00 and robo.opcode & 0xfe != 0xfe and robo.opcode & 0xee != 0x20 and robo.opcode & 0xf7 != 0x30 and robo.opcode & 0xed != 0x20
